In [1]:
# Downgrade pillow to avoid `UserWarning: Possibly corrupt EXIF data.`
!pip install pillow==4.0.0
!pip install keras

    100% |████████████████████████████████| 5.6MB 5.8MB/s eta 0:00:01
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 378kB 30.8MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%ls -la
%cd eye-Image-Analysis/
%ls -la

total 74488
drwxrwxr-x 14 ec2-user ec2-user     4096 Dec 17 12:38 ./
drwxr-xr-x  5 ec2-user ec2-user     4096 Dec 17 12:22 ../
drwxrwxr-x  5 ec2-user ec2-user     4096 Dec 13 01:48 Data/
drwxrwxr-x  4 ec2-user ec2-user     4096 Dec 13 01:48 Database/
drwxrwxr-x  5 ec2-user ec2-user     4096 Dec 13 01:48 Data_equalized/
drwxrwxr-x  3 ec2-user ec2-user     4096 Dec 13 01:48 Data_partial_excluded/
drwxrwxr-x  3 ec2-user ec2-user     4096 Dec 13 01:48 Data_Suspect/
drwxrwxr-x  8 ec2-user ec2-user     4096 Dec 16 20:21 efficientnet_keras_transfer_learning/
-rw-rw-r--  1 ec2-user ec2-user     1487 Dec 13 01:48 evaluation_test.py
drwxrwxr-x  8 ec2-user ec2-user     4096 Dec 17 12:38 .git/
-rw-rw-r--  1 ec2-user ec2-user     1212 Dec 13 01:48 .gitignore
-rw-rw-r--  1 ec2-user ec2-user     4859 Dec 13 01:48 Graphs.py
-rw-rw-r--  1 ec2-user ec2-user    14282 Dec 17 12:36 Interpreter.py
-rw-rw-r--  1 ec2-user ec2-user    11233 Dec 13 02:25 Interpreter.pyc
drwxrwxr-x  2 ec2-user ec2-user     4096 

In [3]:
from Interpreter import Interpreter

Using TensorFlow backend.


In [4]:
TARGET_SIZE = (256, 256)
BATCH_SIZE = 32
EPOCHS = 100
IMAGE_SHAPE = (256, 256, 3)

inter = Interpreter(
    BATCH_SIZE,
    IMAGE_SHAPE,
    EPOCHS,
    TARGET_SIZE
)

train_images, validation_images, test_images = inter.split_data()

Found 706 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Found 94 images belonging to 2 classes.


In [5]:
model, model_out = inter.train_efficient_net(
    train_images,
    test_images,
    validation_images
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
16719872/16717576 [==============================] - 3s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 8, 8, 1280)        4049564   
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 1280)              0         
_________________________________________________________________
dropout_out (Dropout)        (None, 1280)              0         
_________________________________________________________________
fc_out (Dense)               (None, 2

Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/keras/utils/data_utils.py", line 445, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/keras_preprocessing/image/iterator.py", line 65, in __getitem__
    return self._get_batches_of_transformed_samples(index_array)
  File "/home/ec2-user/anaconda3/envs/amazonei_ten

KeyboardInterrupt: 